### Notebook 4 - Post-MCMC parameter estimation, extract parameter estimates from the sampled chains

The chains underlying the parameter estimates in the paper are not contained within this github repository, but instead can be found in the zenodo data repository linked to from the README. The same is true for the figures associated with each gene's parameter estimates (corner plots and fitted profiles). The result summary extracted by this notebook is "extracted_results/MCMC_result_collection.csv".

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from os import listdir
from os.path import isfile, join

# custom functions
from MCMC_pipe import import_data, HDI, protein_ODE

#### A) Get list of files from which to collect data, prepare functions

In [3]:
# identify folder with MCMC chains and posterior predictive results
MC_folder = "RESULTS/MCMC_results/"
MC_chain_list = [f for f in listdir(MC_folder) if isfile(join(MC_folder, f)) and f.endswith('chain_sample.csv')]
MC_pp_list = [f for f in listdir(MC_folder) if isfile(join(MC_folder, f)) and f.endswith('posterior_predictive_results.csv')]

In [4]:
# sort both lists alphabetically for easier iteration
MC_chain_list = sorted(MC_chain_list)
MC_pp_list = sorted(MC_pp_list)
len(MC_chain_list)

2866

In [5]:
# to calculate model traces, we also need the data
M_data, P_data, Psem_data, M_interp_dict, beta_gamma_dist, delta_gamma_dist, start_values = \
    import_data(import_folder='processed_data')

#### B) Go through genes and extract information

In [6]:
# prepare an array to collect all info, a total of 53 data points
res_array = np.zeros((len(MC_chain_list), 54))
gene_list = []

for i, chain_file in enumerate(MC_chain_list):
    if i%100 == 0:
        print(i)
    # prep
    gene = chain_file.split('_')[0]
    gene_list.append(gene)
    res_dict = {}
    # get MCMC chain df
    chain_path = MC_folder + chain_file
    chain_df = pd.read_csv(chain_path, index_col=0)
    # get posterior predictive dict
    pp_path = MC_folder + gene + '_' + 'posterior_predictive_results.csv'
    pp_df = pd.read_csv(pp_path, index_col=0)

    # get the mode for this gene and write to dict
    res_dict['beta_mode'] = np.exp(chain_df.iloc[chain_df['log_prob'].argmax()]['log_beta'])
    res_dict['delta_mode'] = np.exp(chain_df.iloc[chain_df['log_prob'].argmax()]['log_delta'])
    res_dict['Pzero_mode'] = chain_df.iloc[chain_df['log_prob'].argmax()]['Pzero']

    # calculate the MAP profile and write each tp to dict
    time_vec = np.linspace(0, 96, 6)
    mRNA_fun = M_interp_dict[gene]
    protein_vals = P_data.loc[gene].values
    model_vals = odeint(protein_ODE, res_dict['Pzero_mode'], time_vec,
                         args=(mRNA_fun, res_dict['beta_mode'], res_dict['delta_mode'])).T[0]    
    MAP_meannorm = model_vals/np.mean(model_vals)
    for tp in range(6):
        res_dict['protein model MAP (mean-normalized), V{}'.format(tp+1)] = MAP_meannorm[tp]
        res_dict['protein model MAP (scaled to molecules/cell), V{}'.format(tp+1)] = model_vals[tp]

    # 95% highest density intervals
    beta_95_left, beta_95_right = HDI(chain_df['log_beta'], 0.95)
    res_dict['beta_HDI95_left'] = np.exp(beta_95_left)
    res_dict['beta_HDI95_right'] = np.exp(beta_95_right)

    delta_95_left, delta_95_right = HDI(chain_df['log_delta'], 0.95)
    res_dict['delta_HDI95_left'] = np.exp(delta_95_left)
    res_dict['delta_HDI95_right'] = np.exp(delta_95_right)

    # 68% highest density intervals
    beta_68_left, beta_68_right = HDI(chain_df['log_beta'], 0.68)
    res_dict['beta_HDI68_left'] = np.exp(beta_68_left)
    res_dict['beta_HDI68_right'] = np.exp(beta_68_right)

    delta_68_left, delta_68_right = HDI(chain_df['log_delta'], 0.68)
    res_dict['delta_HDI68_left'] = np.exp(delta_68_left)
    res_dict['delta_HDI68_right'] = np.exp(delta_68_right)

    # get median and percentiles for this gene and write to dict
    beta_percs = np.percentile(chain_df['log_beta'], [16, 50, 84])
    delta_percs = np.percentile(chain_df['log_delta'], [16, 50, 84])

    res_dict['beta_p16'] = np.exp(beta_percs[0])
    res_dict['beta_p50'] = np.exp(beta_percs[1])
    res_dict['beta_p84'] = np.exp(beta_percs[2])

    res_dict['delta_p16'] = np.exp(delta_percs[0])
    res_dict['delta_p50'] = np.exp(delta_percs[1])
    res_dict['delta_p84'] = np.exp(delta_percs[2])
    
    # now, add the posterior predictive pvalues
    for modval in pp_df.index:
        res_dict[modval] = pp_df.loc[modval]['value']
    
    # get the values and write them to the array
    res_array[i, :] = [x[1] for x in list(res_dict.items())]
    
# at the end, also get the keys and make dataframe, export
value_names = [x[0] for x in list(res_dict.items())]
res_df = pd.DataFrame(data=res_array, columns=value_names, index=gene_list)
res_df.sort_index(axis=1, inplace=True)
res_df.to_csv('extracted_results/MCMC_result_collection.csv')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
